# Producing the Transfer Functions

In [1]:
%load_ext autoreload
import sys
sys.path.append("../../..")

## Notebook Initialization

In [2]:
%autoreload
import numpy as np
import pickle
from tqdm import tqdm_notebook as tqdm

import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = [10,10]


from darkhistory.spec.spectrum import Spectrum
import darkhistory.spec.spectools as spectools
import darkhistory.spec.transferfunction as tf
import darkhistory.spec.transferfunclist as tflist
import darkhistory.physics as phys
import darkhistory.utilities as utils

from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator

# np.set_printoptions(threshold=np.nan)

## Import Raw Data

In [3]:
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_full_xHe/raw/'

xes_str = ['4.540E-05', '9.952E-03', '2.689E-01', '5.000E-01', '6.900E-01', '9.879E-01', '9.997E-01']

file_names = [[
        direc +'transferfunction_helium_xH_'
        +xH_str+'_xHe_'+xHe_str+'.p'
        for xHe_str in xes_str    
    ] for xH_str in xes_str
]

raw_tf_tuple = [[pickle.load(open(file_name, "rb")) for file_name in file_names_xH] for file_names_xH in file_names]

flt32 = True

if flt32:
    arr_dtype = np.float32
else:
    arr_dtype = np.float64

# xH, xHe, in_eng, rs, eng, type={highengphot, lowengphot, lowengelec}
raw_tf_list = np.array([[tup[0] for tup in raw_tf_tuple_xH] for raw_tf_tuple_xH in raw_tf_tuple], dtype=arr_dtype)
# xH, xHe, in_eng, rs
CMB_engloss_list = np.array([[tup[1] for tup in raw_tf_tuple_xH] for raw_tf_tuple_xH in raw_tf_tuple], dtype=arr_dtype)
# xH, xHe, in_eng, rs, {H-ion, Exc, Heating, Cont.}
highdeposited_list = np.array([[tup[2] for tup in raw_tf_tuple_xH] for raw_tf_tuple_xH in raw_tf_tuple], dtype=arr_dtype)
# xH, xHe, in_eng, rs, type={mwimp, rs}
info_list = np.array([[tup[3] for tup in raw_tf_tuple_xH] for raw_tf_tuple_xH in raw_tf_tuple], dtype=arr_dtype)
# xH, xHe, in_eng, rs, type={photeng, eleceng}, eng
abscissae_list = np.array([[tup[4] for tup in raw_tf_tuple_xH] for raw_tf_tuple_xH in raw_tf_tuple])



In [4]:
print(raw_tf_list.shape)
print(raw_tf_list.dtype)

(7, 7, 327, 20, 500, 3)
float32


In [5]:
# This checks that all of the abscissae are equal.
for i in np.arange(info_list.shape[0]):
    for j in np.arange(info_list.shape[1]):
        for k in np.arange(info_list.shape[2]):
            if not np.array_equal(
                info_list[i,j,k,:,1],
                info_list[0,0,0,:,1]
            ):
                print('Not all the same redshifts!')
            for l in np.arange(info_list.shape[3]):
                if not np.array_equal(
                    info_list[i,j,:,l,0],
                    info_list[0,0,:,0,0]
                ):
                    print('Not all the same mwimps!')
                if not np.array_equal(
                    abscissae_list[i,j,k,l,0,:],
                    abscissae_list[0,0,0,0,0,:]
                ):
                    print('Not all the same photeng abscissa!')
                if not np.array_equal(
                    abscissae_list[i,j,k,l,1,:],
                    abscissae_list[0,0,0,0,1,:]
                ):
                    print('Not all the same eleceng abscissa!')
                
xe_arr = 0.5 + 0.5*np.tanh(
    [4.540e-5, 9.952e-3, 2.689e-1, 5.000e-1, 6.900e-1, 9.879e-1, 9.997e-1]
)

rs_arr    = info_list[0,0,0,:,1]
mwimp_arr = info_list[0,0,:,0,0]
photeng = abscissae_list[0,0,0,0,0,:]
eleceng = abscissae_list[0,0,0,0,1,:]

## Raw Data Manipulation

The transfer function is calculated by injecting 2 photons at the input energy abscissa. However, the first step is to assign $x$ photons to the top bin of the *output* photon abscissa, such that $x$ times the energy of the top bin is 2 times the injection energy. Because of the misalignment between the two abscissae, as well as the fact that we want to transfer function for a single electron, we have to normalize the results first.

First, we construct two arrays: a list of output abscissae (for both photons and electrons) given the injection abscissa, and a list of the energy of the *output* photon energy bin where the injected photons are assigned to. Then we compute the normalization factor. 

In [6]:
# This treatment is consistent with ih_transferfunction
photeng_bin_width = np.diff(np.log(spectools.get_bin_bound(photeng))) * photeng
eleceng_bin_width = np.diff(np.log(spectools.get_bin_bound(eleceng))) * eleceng


# Original injection energies
top_photeng_bins = np.array(
    [
        photeng[photeng <= mwimp][-1]
        for mwimp in mwimp_arr
    ]
)

norm_fac_arr = 2*mwimp_arr/top_photeng_bins

Dividing by the normalization factor gives the spectra for the injection of 1 photon, with energy given by `top_photeng_bins`. 

Now we construct a raw array of all of the transfer functions, on which we will perform an interpolation to extract a set of transfer functions for use, given a chosen abscissa.

In [7]:
# Multiplying by bin width converts all the eleceng array into number of particles. 
# Indexing: xH, xHe, in_eng, rs, eng, type={highengphot, lowengphot, lowengelec}

phot_tf_raw = np.array([
    [
        [
            [
                raw_tf[i,j,:,0]/norm_fac*photeng_bin_width
                    for (j, rs) in enumerate(rs_arr)   
            ]
            for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))

        ] for raw_tf in raw_tf_list_xH
    ] for raw_tf_list_xH in tqdm(raw_tf_list)
], dtype=arr_dtype)

lowengphot_tf_raw = np.array([
    [
        [
            [
                raw_tf[i,j,:,1]/norm_fac*photeng_bin_width 
                    for (j, rs) in enumerate(rs_arr)   
            ]
            for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))

        ] for raw_tf in raw_tf_list_xH
    ] for raw_tf_list_xH in tqdm(raw_tf_list)
], dtype=arr_dtype)

lowengelec_tf_raw = np.array([
    [
        [
            [
                raw_tf[i,j,:,2]/norm_fac*eleceng_bin_width 
                    for (j, rs) in enumerate(rs_arr)   
            ]
            for (i, (in_eng, norm_fac)) in enumerate(zip(mwimp_arr, norm_fac_arr))

        ] for raw_tf in raw_tf_list_xH
    ] for raw_tf_list_xH in tqdm(raw_tf_list)
], dtype=arr_dtype)

# xH, xHe, in_eng, rs
CMB_engloss = CMB_engloss_list/np.outer(norm_fac_arr, np.ones_like(rs_arr))

# xH, xHe, in_eng, rs, {H-Ion, Exc, Heat, Cont}
denom_highdeposited = np.ones((xe_arr.size, xe_arr.size, rs_arr.size, 4, norm_fac_arr.size))
denom_highdeposited *= norm_fac_arr
denom_highdeposited = np.rollaxis(denom_highdeposited, 4, 2)
highdeposited = highdeposited_list/denom_highdeposited

In [8]:
photeng_high = photeng[photeng > 60]
print(photeng_high.shape)
print(lowengelec_tf_raw.dtype)
new_phot_tf_raw = phot_tf_raw
new_lowengphot_tf_raw = lowengphot_tf_raw
new_lowengelec_tf_raw = lowengelec_tf_raw
new_cmbloss_raw = CMB_engloss
new_highdeposited_raw = highdeposited

(327,)
float32


Now, we construct the list of `Spectrum` objects from our raw array of transfer functions...

In [9]:
# spec_type = 'N' for eleceng, because we have already multiplied in the bin width earlier. 

#xH, xHe, in_eng, rs, eng
photspec_list = [
    [
        [
            [
                Spectrum(
                    photeng, phot_tf[i,j,:], spec_type='N', 
                    rs = rs, in_eng = in_eng
                ) for (j, rs) in enumerate(rs_arr)
            ] for (i, in_eng) in enumerate(photeng_high)
        ] for phot_tf in phot_tf_list_xH
    ] for phot_tf_list_xH in tqdm(new_phot_tf_raw)
]

lowengphotspec_list = [
    [
        [
            [
                Spectrum(
                    photeng, lowengphot_tf[i,j,:], spec_type='N', 
                    rs = rs, in_eng = in_eng
                ) for (j, rs) in enumerate(rs_arr)
            ] for (i, in_eng) in enumerate(photeng_high)
        ] for lowengphot_tf in lowengphot_tf_list_xH
    ] for lowengphot_tf_list_xH in tqdm(new_lowengphot_tf_raw)
]

lowengelecspec_list = [
    [
        [
            [
                Spectrum(
                    eleceng, lowengelec_tf[i,j,:], spec_type='N', 
                    rs = rs, in_eng = in_eng
                ) for (j, rs) in enumerate(rs_arr)
            ] for (i, in_eng) in enumerate(photeng_high)
        ] for lowengelec_tf in tqdm(lowengelec_tf_list_xH)
    ] for lowengelec_tf_list_xH in tqdm(new_lowengelec_tf_raw)
]


Finally, we can construct the `TransferFuncList` from this.

In [10]:
tfunclist_photspec_list = [
    [
        tflist.TransferFuncList(
            [
                tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
                for spec_arr in photspec_xH
            ]
        ) for photspec_xH in photspec_list_xH
    ]for photspec_list_xH in tqdm(photspec_list)
]

tfunclist_lowengphotspec_list = [
    [
        tflist.TransferFuncList(
            [
                tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
                for spec_arr in lowengphotspec_xH
            ]
        ) for lowengphotspec_xH in lowengphotspec_list_xH
    ]for lowengphotspec_list_xH in tqdm(lowengphotspec_list)
]
    

tfunclist_lowengelecspec_list = [
    [
        tflist.TransferFuncList(
            [
                tf.TransFuncAtEnergy(spec_arr, dlnz=0.001)
                for spec_arr in lowengelecspec_xH
            ]
        ) for lowengelecspec_xH in lowengelecspec_list_xH
    ] for lowengelecspec_list_xH in tqdm(lowengelecspec_list)
]

We will now transpose all of the `TransferFuncList` objects so that they are now all of type `'rs'`.

In [11]:
for tflists_xH_hep, tflists_xH_lep, tflists_xH_lee in zip(
    tfunclist_photspec_list, tfunclist_lowengphotspec_list, tfunclist_lowengelecspec_list):
    for tflist_hep, tflist_lep, tflist_lee in zip(
        tflists_xH_hep, tflists_xH_lep, tflists_xH_lee
    ):
        tflist_hep.transpose()
        tflist_lep.transpose()
        tflist_lee.transpose()
        for i, tf_hep, tf_lep, tf_lee in zip(np.arange(327), tflist_hep, tflist_lep, tflist_lee):
            if np.any(np.isnan(tf_hep._grid_vals)):
                print(np.where(np.isnan(tf_hep._grid_vals)),'here!')
            if np.any(np.isnan(tf_lep._grid_vals)):
                print(np.where(np.isnan(tf_lep._grid_vals)),'here1!')
            if np.any(np.isnan(tf_lee._grid_vals)):
                print(np.where(np.isnan(tf_lee._grid_vals)),'here2!')

            tf_hep._grid_vals[np.where(tf_hep.grid_vals < 0)] = 1e-100
            tf_lep._grid_vals[np.where(tf_lep.grid_vals < 0)] = 1e-100
            tf_lee._grid_vals[np.where(tf_lee.grid_vals < 0)] = 1e-100


And, finally, save the transfer functions.

In [12]:
def save_as_pickled_object(obj, filepath):
    """
    This is a defensive way to write pickle.write, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])

In [13]:
direc = '/Users/hongwan/Dropbox (MIT)/Photon Deposition/tfs_full_xHe/'
string = '_full_xHe'

if not flt32:

    save_as_pickled_object(tfunclist_photspec_list, direc+"tfunclist_photspec_60eV_complete"+string+".raw")
    save_as_pickled_object(tfunclist_lowengphotspec_list, direc+"tfunclist_lowengphotspec_60eV_complete"+string+".raw")
    save_as_pickled_object(tfunclist_lowengelecspec_list, direc+"tfunclist_lowengelecspec_60eV_complete"+string+".raw")
    save_as_pickled_object(new_cmbloss_raw, direc+"CMB_engloss_60eV_complete"+string+".raw")
    save_as_pickled_object(new_highdeposited_raw, direc+"highdeposited_60eV_complete"+string+".raw")
    
else:

    save_as_pickled_object(tfunclist_photspec_list, direc+"tfunclist_photspec_60eV_complete"+string+"_flt32.raw")
    save_as_pickled_object(tfunclist_lowengphotspec_list, direc+"tfunclist_lowengphotspec_60eV_complete"+string+"_flt32.raw")
    save_as_pickled_object(tfunclist_lowengelecspec_list, direc+"tfunclist_lowengelecspec_60eV_complete"+string+"_flt32.raw")
    save_as_pickled_object(new_cmbloss_raw, direc+"CMB_engloss_60eV_complete"+string+"_flt32.raw")
    save_as_pickled_object(new_highdeposited_raw, direc+"highdeposited_60eV_complete"+string+"_flt32.raw")
    
    

In [1]:
tfunclist_photspec_list[0][0].grid_vals.dtype


NameError: name 'tfunclist_photspec_list' is not defined

In [15]:
flt32

True